In [4]:
import pandas as pd
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

In [5]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": [32.806671, 61.370716, 33.729759, 34.969704, 36.116203, 39.059811, 41.597782, 39.318523, 38.897438,
                 27.766279, 33.040619, 21.094318, 44.240459, 40.349457, 39.849426, 42.011539, 38.5266, 37.66814,
                 31.169546, 44.693947, 39.063946, 42.230171, 43.326618, 45.694454, 32.741646, 38.456085, 46.921925,
                 41.12537, 38.313515, 43.452492, 40.298904, 34.840515, 42.165726, 35.630066, 47.528912, 40.388783,
                 35.565342, 44.572021, 40.590752, 41.680893, 33.856892, 44.299782, 35.747845, 31.054487, 40.150032,
                 44.045876, 37.769337, 47.400902, 38.491226, 44.268543, 42.755966],
    "longitude": [-86.79113, -152.404419, -111.431221, -92.373123, -119.681564, -105.311104, -72.755371, -75.507141,
                  -77.026817, -81.686783, -83.643074, -157.498337, -114.478828, -88.986137, -86.258278, -93.210526,
                  -96.726486, -84.670067, -91.867805, -69.381927, -76.802101, -71.530106, -84.536095, -93.900192,
                  -89.678696, -92.288368, -110.454353, -98.268082, -117.055374, -71.563896, -74.521011, -106.248482,
                  -74.948051, -79.806419, -99.784012, -82.764915, -96.928917, -122.070938, -77.209755, -71.51178,
                  -80.945007, -99.438828, -86.692345, -97.563461, -111.862434, -72.710686, -78.169968, -121.490494,
                  -80.954453, -89.616508, -107.30249],
    "start_date": "2000-01-31",
    "end_date": "2023-12-31",
    "hourly": "temperature_2m",
    "daily": ["temperature_2m_max", "temperature_2m_min", "daylight_duration", "rain_sum", "snowfall_sum"],
    "temperature_unit": "fahrenheit",
    "wind_speed_unit": "mph",
    "precipitation_unit": "inch"
}
responses = openmeteo.weather_api(url, params=params)

# Initialize lists to hold hourly and daily data
hourly_dataframes = []
daily_dataframes = []

# Loop through all responses to process data for each location
for response in responses:
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_data = {
        "date": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        ),
        "latitude": response.Latitude(),
        "longitude": response.Longitude(),
        "temperature_2m": hourly_temperature_2m
    }
    hourly_dataframes.append(pd.DataFrame(data=hourly_data))

    daily = response.Daily()
    daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
    daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
    daily_daylight_duration = daily.Variables(2).ValuesAsNumpy()
    daily_rain_sum = daily.Variables(3).ValuesAsNumpy()
    daily_snowfall_sum = daily.Variables(4).ValuesAsNumpy()

    daily_data = {
        "date": pd.date_range(
            start=pd.to_datetime(daily.Time(), unit="s", utc=True),
            end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=daily.Interval()),
            inclusive="left"
        ),
        "latitude": response.Latitude(),
        "longitude": response.Longitude(),
        "temp_max": daily_temperature_2m_max,
        "temp_min": daily_temperature_2m_min,
        
        "daylight": daily_daylight_duration,
        "rain_sum": daily_rain_sum,
        "snowfall_sum": daily_snowfall_sum


    }
    daily_dataframes.append(pd.DataFrame(data=daily_data))

# Concatenate hourly and daily dataframes for all locations
hourly_dataframe = pd.concat(hourly_dataframes, ignore_index=True)
daily_dataframe = pd.concat(daily_dataframes, ignore_index=True)

# print(hourly_dataframe)
# print(daily_dataframe)


In [6]:
daily_dataframe

,date,latitude,longitude,temp_max,temp_min,daylight,rain_sum,snowfall_sum
0,2000-01-31 00:00:00+00:00,32.794376,-86.809296,42.799099,27.679100,38088.992188,0.0,0.0
1,2000-02-01 00:00:00+00:00,32.794376,-86.809296,51.619099,28.489100,38185.601562,0.0,0.0
2,2000-02-02 00:00:00+00:00,32.794376,-86.809296,49.189098,29.569099,38283.316406,0.0,0.0
3,2000-02-03 00:00:00+00:00,32.794376,-86.809296,53.599098,27.589100,38382.019531,0.0,0.0
4,2000-02-04 00:00:00+00:00,32.794376,-86.809296,52.429100,35.869099,38481.609375,0.0,0.0
...,...,...,...,...,...,...,...,...
445531,2023-12-27 00:00:00+00:00,42.776798,-107.440826,20.932699,6.802700,32558.552734,0.0,0.0
445532,2023-12-28 00:00:00+00:00,42.776798,-107.440826,19.042700,6.982700,32585.277344,0.0,0.0
445533,2023-12-29 00:00:00+00:00,42.776798,-107.440826,22.372700,12.202702,32616.460938,0.0,0.0
445534,2023-12-30 00:00:00+00:00,42.776798,-107.440826,25.522699,15.622700,32652.011719,0.0,0.0


In [7]:
daily_dataframe['lat'] = daily_dataframe['latitude'].astype(str).str[:2]
daily_dataframe['lon'] = daily_dataframe['longitude'].astype(str).str[:3]
daily_dataframe['concat'] = daily_dataframe['lat']+daily_dataframe['lon']
daily_dataframe

,date,latitude,longitude,temp_max,temp_min,daylight,rain_sum,snowfall_sum,lat,lon,concat
0,2000-01-31 00:00:00+00:00,32.794376,-86.809296,42.799099,27.679100,38088.992188,0.0,0.0,32,-86,32-86
1,2000-02-01 00:00:00+00:00,32.794376,-86.809296,51.619099,28.489100,38185.601562,0.0,0.0,32,-86,32-86
2,2000-02-02 00:00:00+00:00,32.794376,-86.809296,49.189098,29.569099,38283.316406,0.0,0.0,32,-86,32-86
3,2000-02-03 00:00:00+00:00,32.794376,-86.809296,53.599098,27.589100,38382.019531,0.0,0.0,32,-86,32-86
4,2000-02-04 00:00:00+00:00,32.794376,-86.809296,52.429100,35.869099,38481.609375,0.0,0.0,32,-86,32-86
...,...,...,...,...,...,...,...,...,...,...,...
445531,2023-12-27 00:00:00+00:00,42.776798,-107.440826,20.932699,6.802700,32558.552734,0.0,0.0,42,-10,42-10
445532,2023-12-28 00:00:00+00:00,42.776798,-107.440826,19.042700,6.982700,32585.277344,0.0,0.0,42,-10,42-10
445533,2023-12-29 00:00:00+00:00,42.776798,-107.440826,22.372700,12.202702,32616.460938,0.0,0.0,42,-10,42-10
445534,2023-12-30 00:00:00+00:00,42.776798,-107.440826,25.522699,15.622700,32652.011719,0.0,0.0,42,-10,42-10


In [8]:
state_merge = pd.read_csv("C:/Github/Group3_Project4/pete files/pete_resources/state lat lon.csv")
state_merge.head()

,State,State Abbr,Latitude,Longitude,reduced,reduced.1,concat,region,sub-region,country
0,Alabama,AL,32.806671,-86.791130,32,-86,32-86,South,East South Central,US
1,Alaska,AK,61.370716,-152.404419,61,-15,61-15,West,Pacific,US
2,Arizona,AZ,33.729759,-111.431221,33,-11,33-11,West,Mountain,US
3,Arkansas,AR,34.969704,-92.373123,34,-92,34-92,South,West South Central,US
4,California,CA,36.116203,-119.681564,36,-11,36-11,West,Pacific,US


In [9]:
merged_weather = pd.merge(daily_dataframe, state_merge, on = 'concat', how = 'left')
merged_weather.head()

,date,latitude,longitude,temp_max,temp_min,daylight,rain_sum,snowfall_sum,lat,lon,concat,State,State Abbr,Latitude,Longitude,reduced,reduced.1,region,sub-region,country
0,2000-01-31 00:00:00+00:00,32.794376,-86.809296,42.799099,27.679100,38088.992188,0.0,0.0,32,-86,32-86,Alabama,AL,32.806671,-86.79113,32,-86,South,East South Central,US
1,2000-02-01 00:00:00+00:00,32.794376,-86.809296,51.619099,28.489100,38185.601562,0.0,0.0,32,-86,32-86,Alabama,AL,32.806671,-86.79113,32,-86,South,East South Central,US
2,2000-02-02 00:00:00+00:00,32.794376,-86.809296,49.189098,29.569099,38283.316406,0.0,0.0,32,-86,32-86,Alabama,AL,32.806671,-86.79113,32,-86,South,East South Central,US
3,2000-02-03 00:00:00+00:00,32.794376,-86.809296,53.599098,27.589100,38382.019531,0.0,0.0,32,-86,32-86,Alabama,AL,32.806671,-86.79113,32,-86,South,East South Central,US
4,2000-02-04 00:00:00+00:00,32.794376,-86.809296,52.429100,35.869099,38481.609375,0.0,0.0,32,-86,32-86,Alabama,AL,32.806671,-86.79113,32,-86,South,East South Central,US


In [10]:
weather = merged_weather.drop(columns = ['lat', 'lon', 'concat', 'Latitude', 'Longitude', 'reduced','reduced.1'])
# weather.set_index('State', inplace=True)
weather


,date,latitude,longitude,temp_max,temp_min,daylight,rain_sum,snowfall_sum,State,State Abbr,region,sub-region,country
0,2000-01-31 00:00:00+00:00,32.794376,-86.809296,42.799099,27.679100,38088.992188,0.0,0.0,Alabama,AL,South,East South Central,US
1,2000-02-01 00:00:00+00:00,32.794376,-86.809296,51.619099,28.489100,38185.601562,0.0,0.0,Alabama,AL,South,East South Central,US
2,2000-02-02 00:00:00+00:00,32.794376,-86.809296,49.189098,29.569099,38283.316406,0.0,0.0,Alabama,AL,South,East South Central,US
3,2000-02-03 00:00:00+00:00,32.794376,-86.809296,53.599098,27.589100,38382.019531,0.0,0.0,Alabama,AL,South,East South Central,US
4,2000-02-04 00:00:00+00:00,32.794376,-86.809296,52.429100,35.869099,38481.609375,0.0,0.0,Alabama,AL,South,East South Central,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...
445531,2023-12-27 00:00:00+00:00,42.776798,-107.440826,20.932699,6.802700,32558.552734,0.0,0.0,Wyoming,WY,West,Mountain,US
445532,2023-12-28 00:00:00+00:00,42.776798,-107.440826,19.042700,6.982700,32585.277344,0.0,0.0,Wyoming,WY,West,Mountain,US
445533,2023-12-29 00:00:00+00:00,42.776798,-107.440826,22.372700,12.202702,32616.460938,0.0,0.0,Wyoming,WY,West,Mountain,US
445534,2023-12-30 00:00:00+00:00,42.776798,-107.440826,25.522699,15.622700,32652.011719,0.0,0.0,Wyoming,WY,West,Mountain,US


In [11]:
# separate out the month and day from the date column
weather['month'] = weather['date'].dt.month
weather['year'] = weather['date'].dt.year
weather['Date'] =  weather['month'].astype(str).str.zfill(2) + '-' + weather['year'].astype(str)
weather['Date'] = pd.to_datetime(weather['Date'])
changed_date_weather = weather.drop(columns = ['date']) #keep lat and lon for tableau mapping?
changed_date_weather

C:\Users\peter\AppData\Local\Temp\ipykernel_4988\4041428879.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  weather['Date'] = pd.to_datetime(weather['Date'])


,latitude,longitude,temp_max,temp_min,daylight,rain_sum,snowfall_sum,State,State Abbr,region,sub-region,country,month,year,Date
0,32.794376,-86.809296,42.799099,27.679100,38088.992188,0.0,0.0,Alabama,AL,South,East South Central,US,1,2000,2000-01-01
1,32.794376,-86.809296,51.619099,28.489100,38185.601562,0.0,0.0,Alabama,AL,South,East South Central,US,2,2000,2000-02-01
2,32.794376,-86.809296,49.189098,29.569099,38283.316406,0.0,0.0,Alabama,AL,South,East South Central,US,2,2000,2000-02-01
3,32.794376,-86.809296,53.599098,27.589100,38382.019531,0.0,0.0,Alabama,AL,South,East South Central,US,2,2000,2000-02-01
4,32.794376,-86.809296,52.429100,35.869099,38481.609375,0.0,0.0,Alabama,AL,South,East South Central,US,2,2000,2000-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445531,42.776798,-107.440826,20.932699,6.802700,32558.552734,0.0,0.0,Wyoming,WY,West,Mountain,US,12,2023,2023-12-01
445532,42.776798,-107.440826,19.042700,6.982700,32585.277344,0.0,0.0,Wyoming,WY,West,Mountain,US,12,2023,2023-12-01
445533,42.776798,-107.440826,22.372700,12.202702,32616.460938,0.0,0.0,Wyoming,WY,West,Mountain,US,12,2023,2023-12-01
445534,42.776798,-107.440826,25.522699,15.622700,32652.011719,0.0,0.0,Wyoming,WY,West,Mountain,US,12,2023,2023-12-01


In [12]:
# Annual averages
annual_weather = changed_date_weather

In [13]:
annual_weather['max_temp_avg'] = annual_weather.groupby(['Date', 'State'])['temp_max'].transform('mean')
annual_weather['min_temp_avg'] = annual_weather.groupby(['Date', 'State'])['temp_min'].transform('mean')
annual_weather['daylight_hr_avg'] = annual_weather.groupby(['Date', 'State'])['daylight'].transform('mean')/3600
annual_weather['rain_avg'] = annual_weather.groupby(['Date', 'State'])['rain_sum'].transform('mean')
annual_weather['snowfall_avg'] = annual_weather.groupby(['Date', 'State'])['snowfall_sum'].transform('mean')

annual_weather = annual_weather.drop(columns = ['temp_max', 'temp_min','daylight','rain_sum','snowfall_sum'])
weather_no_dups = annual_weather.drop_duplicates()
weather_no_dups

,latitude,longitude,State,State Abbr,region,sub-region,country,month,year,Date,max_temp_avg,min_temp_avg,daylight_hr_avg,rain_avg,snowfall_avg
0,32.794376,-86.809296,Alabama,AL,South,East South Central,US,1,2000,2000-01-01,42.799099,27.679100,10.580276,0.000000,0.000000
1,32.794376,-86.809296,Alabama,AL,South,East South Central,US,2,2000,2000-02-01,63.412201,44.012547,11.021020,0.056476,0.000000
30,32.794376,-86.809296,Alabama,AL,South,East South Central,US,3,2000,2000-03-01,69.305557,50.759747,12.002685,0.154178,0.000000
61,32.794376,-86.809296,Alabama,AL,South,East South Central,US,4,2000,2000-04-01,72.157097,52.732101,13.012898,0.098688,0.000000
91,32.794376,-86.809296,Alabama,AL,South,East South Central,US,5,2000,2000-05-01,85.528778,66.152649,13.857250,0.042672,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445383,42.776798,-107.440826,Wyoming,WY,West,Mountain,US,8,2023,2023-08-01,73.147217,52.034958,13.889099,0.060960,0.000000
445414,42.776798,-107.440826,Wyoming,WY,West,Mountain,US,9,2023,2023-09-01,67.060699,45.856697,12.498415,0.033727,0.004593
445444,42.776798,-107.440826,Wyoming,WY,West,Mountain,US,10,2023,2023-10-01,48.205601,31.361086,11.051043,0.026670,0.451613
445475,42.776798,-107.440826,Wyoming,WY,West,Mountain,US,11,2023,2023-11-01,35.416698,21.958700,9.770588,0.000262,0.200262


In [14]:
weather_no_dups.to_csv("pete_resources/annual_weather_avgs_2000-2023.csv")

In [15]:
ac_annual_weather = changed_date_weather.drop(columns = ['Date', 'month'])

ac_annual_weather['max_temp_avg'] = ac_annual_weather.groupby(['year'])['temp_max'].transform('mean')
ac_annual_weather['min_temp_avg'] = ac_annual_weather.groupby(['year'])['temp_min'].transform('mean')
ac_annual_weather['daylight_hr_avg'] = ac_annual_weather.groupby(['year'])['daylight'].transform('mean')/3600
ac_annual_weather['rain_avg'] = ac_annual_weather.groupby(['year'])['rain_sum'].transform('mean')
ac_annual_weather['snowfall_avg'] = ac_annual_weather.groupby(['year'])['snowfall_sum'].transform('mean')

ac_annual_weather = ac_annual_weather.drop(columns = ['temp_max', 'temp_min','daylight','rain_sum','snowfall_sum'])
ac_weather_no_dups = ac_annual_weather.drop_duplicates()
ac_weather_no_dups

,latitude,longitude,State,State Abbr,region,sub-region,country,year,max_temp_avg,min_temp_avg,daylight_hr_avg,rain_avg,snowfall_avg
0,32.794376,-86.809296,Alabama,AL,South,East South Central,US,2000,63.630772,46.118309,12.424892,0.080413,0.087108
336,32.794376,-86.809296,Alabama,AL,South,East South Central,US,2001,62.343380,45.102131,12.206055,0.078176,0.099703
701,32.794376,-86.809296,Alabama,AL,South,East South Central,US,2002,62.450317,45.192238,12.206045,0.082099,0.106556
1066,32.794376,-86.809296,Alabama,AL,South,East South Central,US,2003,61.470879,44.427094,12.206116,0.090982,0.109380
1431,32.794376,-86.809296,Alabama,AL,South,East South Central,US,2004,61.573502,44.856514,12.198321,0.094234,0.085110
...,...,...,...,...,...,...,...,...,...,...,...,...,...
443710,42.776798,-107.440826,Wyoming,WY,West,Mountain,US,2019,62.175228,44.758846,12.206122,0.103378,0.133776
444075,42.776798,-107.440826,Wyoming,WY,West,Mountain,US,2020,63.667397,46.003712,12.198345,0.100059,0.109559
444441,42.776798,-107.440826,Wyoming,WY,West,Mountain,US,2021,63.684147,46.113163,12.206074,0.100798,0.111712
444806,42.776798,-107.440826,Wyoming,WY,West,Mountain,US,2022,63.473381,44.741516,12.205966,0.096211,0.127173


In [16]:
#TEN YEAR FILTER 
filter_start = '2011-01-01'
filter_end = '2020-12-31' 

# month_filter_start = 1
# month_filter_end = 12

year_begin = (weather_no_dups['Date'] >= filter_start) #(weather_no_dups['month'] >= month_filter_start)

year_end = (weather_no_dups['Date'] <= filter_end) #(weather_no_dups['month'] <= month_filter_end)

# Apply the filters
ten_yr_filtered_weather = weather_no_dups.loc[year_begin & year_end]

ten_yr_filtered_weather.head()

,latitude,longitude,State,State Abbr,region,sub-region,country,month,year,Date,max_temp_avg,min_temp_avg,daylight_hr_avg,rain_avg,snowfall_avg
3988,32.794376,-86.809296,Alabama,AL,South,East South Central,US,1,2011,2011-01-01,51.189423,35.770390,10.266394,0.103124,0.043561
4019,32.794376,-86.809296,Alabama,AL,South,East South Central,US,2,2011,2011-02-01,60.876244,42.516243,11.015700,0.142998,0.025591
4047,32.794376,-86.809296,Alabama,AL,South,East South Central,US,3,2011,2011-03-01,68.896202,50.161678,11.980432,0.209169,0.000000
4078,32.794376,-86.809296,Alabama,AL,South,East South Central,US,4,2011,2011-04-01,78.568100,57.991100,12.992403,0.138583,0.000000
4108,32.794376,-86.809296,Alabama,AL,South,East South Central,US,5,2011,2011-05-01,83.452965,61.794907,13.842589,0.031623,0.000000


In [17]:

# ten_yr_filtered_weather.to_excel("pete_resources/10yr_home_and_weather.xlsx")
# ten_yr_filtered_weather.to_csv("pete_resources/10yr_home_and_weather.csv")
# ten_yr_filtered_weather.dtypes

In [18]:

energy_data = pd.read_csv("C:/Github/Group3_Project4/pete files/pete_resources/all_energy_plus_all_renewables.csv")
energy_data = energy_data.drop(columns = 'Unnamed: 0')
energy_data = energy_data.rename(columns = {'State': 'State Abbr'})
energy_data.head()


,State Abbr,2011(C),2012(C),2013(C),2014(C),2015(C),2016(C),2017(C),2018(C),2019(C),...,2011(W),2012(W),2013(W),2014(W),2015(W),2016(W),2017(W),2018(W),2019(W),2020(W)
0,AK,15481,15521,14819,18225,19511,16613,16376,17325,17620,...,121,356,1386,1445,1487,1562,1305,1410,1269,1127
1,AL,651032,547004,565051,575912,494311,410168,378890,377152,317180,...,0,0,0,0,0,0,0,0,0,0
2,AR,306119,296732,327099,339214,226889,246437,267591,304145,239798,...,0,0,0,0,0,0,0,0,0,0
3,AZ,459909,420570,454865,447849,385822,323878,334512,331492,257727,...,2483,5059,4291,4451,4210,4998,5248,4824,4933,5647
4,CA,55264,43832,38151,39486,30967,32077,33663,33256,30886,...,75317,92821,122335,123546,113906,124661,118074,127590,122228,119083


In [25]:
weather_energy_merged = pd.merge(ten_yr_filtered_weather, energy_data, on = 'State Abbr', how = 'left')
weather_energy_merged['2011-(Fossil)'] = weather_energy_merged['2011(N)'] + weather_energy_merged['2011(C)'] + weather_energy_merged['2011(P)'] + weather_energy_merged['2011(G)'] 
weather_energy_merged['2012-(Fossil)'] = weather_energy_merged['2012(N)'] + weather_energy_merged['2012(C)'] + weather_energy_merged['2012(P)'] + weather_energy_merged['2012(G)']
weather_energy_merged['2013-(Fossil)'] = weather_energy_merged['2013(N)'] + weather_energy_merged['2013(C)'] + weather_energy_merged['2013(P)'] + weather_energy_merged['2013(G)']
weather_energy_merged['2014-(Fossil)'] = weather_energy_merged['2014(N)'] + weather_energy_merged['2014(C)'] + weather_energy_merged['2014(P)'] + weather_energy_merged['2014(G)']
weather_energy_merged['2015-(Fossil)'] = weather_energy_merged['2015(N)'] + weather_energy_merged['2015(C)'] + weather_energy_merged['2015(P)'] + weather_energy_merged['2015(G)']
weather_energy_merged['2016-(Fossil)'] = weather_energy_merged['2016(N)'] + weather_energy_merged['2016(C)'] + weather_energy_merged['2016(P)'] + weather_energy_merged['2016(G)']
weather_energy_merged['2017-(Fossil)'] = weather_energy_merged['2017(N)'] + weather_energy_merged['2017(C)'] + weather_energy_merged['2017(P)'] + weather_energy_merged['2017(G)']
weather_energy_merged['2018-(Fossil)'] = weather_energy_merged['2018(N)'] + weather_energy_merged['2018(C)'] + weather_energy_merged['2018(P)'] + weather_energy_merged['2018(G)']
weather_energy_merged['2019-(Fossil)'] = weather_energy_merged['2019(N)'] + weather_energy_merged['2019(C)'] + weather_energy_merged['2019(P)'] + weather_energy_merged['2019(G)']
weather_energy_merged['2020-(Fossil)'] = weather_energy_merged['2020(N)'] + weather_energy_merged['2020(C)'] + weather_energy_merged['2020(P)'] + weather_energy_merged['2020(G)']

weather_energy_merged

,latitude,longitude,State,State Abbr,region,sub-region,country,month,year,Date,...,2011-(Fossil),2012-(Fossil),2013-(Fossil),2014-(Fossil),2015-(Fossil),2016-(Fossil),2017-(Fossil),2018-(Fossil),2019-(Fossil),2020-(Fossil)
0,32.794376,-86.809296,Alabama,AL,South,East South Central,US,1,2011,2011-01-01,...,2193479,2166352,2109702,2148178,2146363,2081488,2036565,2079617,2051095,1952366
1,32.794376,-86.809296,Alabama,AL,South,East South Central,US,2,2011,2011-02-01,...,2193479,2166352,2109702,2148178,2146363,2081488,2036565,2079617,2051095,1952366
2,32.794376,-86.809296,Alabama,AL,South,East South Central,US,3,2011,2011-03-01,...,2193479,2166352,2109702,2148178,2146363,2081488,2036565,2079617,2051095,1952366
3,32.794376,-86.809296,Alabama,AL,South,East South Central,US,4,2011,2011-04-01,...,2193479,2166352,2109702,2148178,2146363,2081488,2036565,2079617,2051095,1952366
4,32.794376,-86.809296,Alabama,AL,South,East South Central,US,5,2011,2011-05-01,...,2193479,2166352,2109702,2148178,2146363,2081488,2036565,2079617,2051095,1952366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6115,42.776798,-107.440826,Wyoming,WY,West,Mountain,US,8,2020,2020-08-01,...,797770,820795,838495,802580,771987,742417,770621,790435,737688,695810
6116,42.776798,-107.440826,Wyoming,WY,West,Mountain,US,9,2020,2020-09-01,...,797770,820795,838495,802580,771987,742417,770621,790435,737688,695810
6117,42.776798,-107.440826,Wyoming,WY,West,Mountain,US,10,2020,2020-10-01,...,797770,820795,838495,802580,771987,742417,770621,790435,737688,695810
6118,42.776798,-107.440826,Wyoming,WY,West,Mountain,US,11,2020,2020-11-01,...,797770,820795,838495,802580,771987,742417,770621,790435,737688,695810


In [31]:
energy_totals_df = weather_energy_merged[['latitude', 'longitude', 'State', 'State Abbr', 'region', 'sub-region',
                                           'country', 'month', 'year', 'Date', '2011-(Fossil)', '2012-(Fossil)',
                                           '2013-(Fossil)', '2014-(Fossil)', '2015-(Fossil)', '2016-(Fossil)', 
                                           '2017-(Fossil)','2018-(Fossil)', '2019-(Fossil)', '2020-(Fossil)',
                                           '2011(TR)','2012(TR)', '2013(TR)', '2014(TR)', '2015(TR)','2016(TR)',
                                           '2017(TR)', '2018(TR)', '2019(TR)', '2020(TR)']]
energy_totals_df

,latitude,longitude,State,State Abbr,region,sub-region,country,month,year,Date,...,2011(TR),2012(TR),2013(TR),2014(TR),2015(TR),2016(TR),2017(TR),2018(TR),2019(TR),2020(TR)
0,32.794376,-86.809296,Alabama,AL,South,East South Central,US,1,2011,2011-01-01,...,279972,265964,336863,293890,284365,259513,285081,301170,298545,308209
1,32.794376,-86.809296,Alabama,AL,South,East South Central,US,2,2011,2011-02-01,...,279972,265964,336863,293890,284365,259513,285081,301170,298545,308209
2,32.794376,-86.809296,Alabama,AL,South,East South Central,US,3,2011,2011-03-01,...,279972,265964,336863,293890,284365,259513,285081,301170,298545,308209
3,32.794376,-86.809296,Alabama,AL,South,East South Central,US,4,2011,2011-04-01,...,279972,265964,336863,293890,284365,259513,285081,301170,298545,308209
4,32.794376,-86.809296,Alabama,AL,South,East South Central,US,5,2011,2011-05-01,...,279972,265964,336863,293890,284365,259513,285081,301170,298545,308209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6115,42.776798,-107.440826,Wyoming,WY,West,Mountain,US,8,2020,2020-08-01,...,61810,55484,54820,56015,52372,58491,59425,54920,57094,68407
6116,42.776798,-107.440826,Wyoming,WY,West,Mountain,US,9,2020,2020-09-01,...,61810,55484,54820,56015,52372,58491,59425,54920,57094,68407
6117,42.776798,-107.440826,Wyoming,WY,West,Mountain,US,10,2020,2020-10-01,...,61810,55484,54820,56015,52372,58491,59425,54920,57094,68407
6118,42.776798,-107.440826,Wyoming,WY,West,Mountain,US,11,2020,2020-11-01,...,61810,55484,54820,56015,52372,58491,59425,54920,57094,68407


In [20]:
weather_energy_merged.to_csv("all_weather_all_energy.csv")
weather_energy_merged.to_excel("all_weather_all_energy.xlsx")

In [33]:
annual_energy_totals = pd.merge(ac_weather_no_dups, energy_data, on = 'State Abbr', how = 'left')
annual_energy_totals['2011-(Fossil)'] = annual_energy_totals['2011(N)'] + annual_energy_totals['2011(C)'] + annual_energy_totals['2011(P)'] + annual_energy_totals['2011(G)'] 
annual_energy_totals['2012-(Fossil)'] = annual_energy_totals['2012(N)'] + annual_energy_totals['2012(C)'] + annual_energy_totals['2012(P)'] + annual_energy_totals['2012(G)']
annual_energy_totals['2013-(Fossil)'] = annual_energy_totals['2013(N)'] + annual_energy_totals['2013(C)'] + annual_energy_totals['2013(P)'] + annual_energy_totals['2013(G)']
annual_energy_totals['2014-(Fossil)'] = annual_energy_totals['2014(N)'] + annual_energy_totals['2014(C)'] + annual_energy_totals['2014(P)'] + annual_energy_totals['2014(G)']
annual_energy_totals['2015-(Fossil)'] = annual_energy_totals['2015(N)'] + annual_energy_totals['2015(C)'] + annual_energy_totals['2015(P)'] + annual_energy_totals['2015(G)']
annual_energy_totals['2016-(Fossil)'] = annual_energy_totals['2016(N)'] + annual_energy_totals['2016(C)'] + annual_energy_totals['2016(P)'] + annual_energy_totals['2016(G)']
annual_energy_totals['2017-(Fossil)'] = annual_energy_totals['2017(N)'] + annual_energy_totals['2017(C)'] + annual_energy_totals['2017(P)'] + annual_energy_totals['2017(G)']
annual_energy_totals['2018-(Fossil)'] = annual_energy_totals['2018(N)'] + annual_energy_totals['2018(C)'] + annual_energy_totals['2018(P)'] + annual_energy_totals['2018(G)']
annual_energy_totals['2019-(Fossil)'] = annual_energy_totals['2019(N)'] + annual_energy_totals['2019(C)'] + annual_energy_totals['2019(P)'] + annual_energy_totals['2019(G)']
annual_energy_totals['2020-(Fossil)'] = annual_energy_totals['2020(N)'] + annual_energy_totals['2020(C)'] + annual_energy_totals['2020(P)'] + annual_energy_totals['2020(G)']

annual_energy_totals.head()

,latitude,longitude,State,State Abbr,region,sub-region,country,year,max_temp_avg,min_temp_avg,...,2011-(Fossil),2012-(Fossil),2013-(Fossil),2014-(Fossil),2015-(Fossil),2016-(Fossil),2017-(Fossil),2018-(Fossil),2019-(Fossil),2020-(Fossil)
0,32.794376,-86.809296,Alabama,AL,South,East South Central,US,2000,63.630772,46.118309,...,2193479,2166352,2109702,2148178,2146363,2081488,2036565,2079617,2051095,1952366
1,32.794376,-86.809296,Alabama,AL,South,East South Central,US,2001,62.343380,45.102131,...,2193479,2166352,2109702,2148178,2146363,2081488,2036565,2079617,2051095,1952366
2,32.794376,-86.809296,Alabama,AL,South,East South Central,US,2002,62.450317,45.192238,...,2193479,2166352,2109702,2148178,2146363,2081488,2036565,2079617,2051095,1952366
3,32.794376,-86.809296,Alabama,AL,South,East South Central,US,2003,61.470879,44.427094,...,2193479,2166352,2109702,2148178,2146363,2081488,2036565,2079617,2051095,1952366
4,32.794376,-86.809296,Alabama,AL,South,East South Central,US,2004,61.573502,44.856514,...,2193479,2166352,2109702,2148178,2146363,2081488,2036565,2079617,2051095,1952366


In [35]:
ac_energy_totals_df = annual_energy_totals[['latitude', 'longitude', 'State', 'State Abbr', 'region', 'sub-region',
                                           'country', 'year', '2011-(Fossil)', '2012-(Fossil)',
                                           '2013-(Fossil)', '2014-(Fossil)', '2015-(Fossil)', '2016-(Fossil)', 
                                           '2017-(Fossil)','2018-(Fossil)', '2019-(Fossil)', '2020-(Fossil)',
                                           '2011(TR)','2012(TR)', '2013(TR)', '2014(TR)', '2015(TR)','2016(TR)',
                                           '2017(TR)', '2018(TR)', '2019(TR)', '2020(TR)']]
ac_energy_totals_df

,latitude,longitude,State,State Abbr,region,sub-region,country,year,2011-(Fossil),2012-(Fossil),...,2011(TR),2012(TR),2013(TR),2014(TR),2015(TR),2016(TR),2017(TR),2018(TR),2019(TR),2020(TR)
0,32.794376,-86.809296,Alabama,AL,South,East South Central,US,2000,2193479,2166352,...,279972,265964,336863,293890,284365,259513,285081,301170,298545,308209
1,32.794376,-86.809296,Alabama,AL,South,East South Central,US,2001,2193479,2166352,...,279972,265964,336863,293890,284365,259513,285081,301170,298545,308209
2,32.794376,-86.809296,Alabama,AL,South,East South Central,US,2002,2193479,2166352,...,279972,265964,336863,293890,284365,259513,285081,301170,298545,308209
3,32.794376,-86.809296,Alabama,AL,South,East South Central,US,2003,2193479,2166352,...,279972,265964,336863,293890,284365,259513,285081,301170,298545,308209
4,32.794376,-86.809296,Alabama,AL,South,East South Central,US,2004,2193479,2166352,...,279972,265964,336863,293890,284365,259513,285081,301170,298545,308209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219,42.776798,-107.440826,Wyoming,WY,West,Mountain,US,2019,797770,820795,...,61810,55484,54820,56015,52372,58491,59425,54920,57094,68407
1220,42.776798,-107.440826,Wyoming,WY,West,Mountain,US,2020,797770,820795,...,61810,55484,54820,56015,52372,58491,59425,54920,57094,68407
1221,42.776798,-107.440826,Wyoming,WY,West,Mountain,US,2021,797770,820795,...,61810,55484,54820,56015,52372,58491,59425,54920,57094,68407
1222,42.776798,-107.440826,Wyoming,WY,West,Mountain,US,2022,797770,820795,...,61810,55484,54820,56015,52372,58491,59425,54920,57094,68407


In [36]:
ac_energy_totals_df.to_csv("summarized_energy_and_weather.csv")
ac_energy_totals_df.to_excel("summarized_energy_and_weather.xlsx")

In [21]:
ac_weather_no_dups.dtypes

latitude           float64
longitude          float64
State               object
State Abbr          object
region              object
sub-region          object
country             object
year                 int32
max_temp_avg       float32
min_temp_avg       float32
daylight_hr_avg    float32
rain_avg           float32
snowfall_avg       float32
dtype: object

In [22]:
#TEN YEAR FILTER 
filter_start = 2011
filter_end = 2020 

# month_filter_start = 1
# month_filter_end = 12

year_begin = (ac_weather_no_dups['year'] >= filter_start) #(weather_no_dups['month'] >= month_filter_start)

year_end = (ac_weather_no_dups['year'] <= filter_end) #(weather_no_dups['month'] <= month_filter_end)

# Apply the filters
ac_ten_yr_filtered_weather = ac_weather_no_dups.loc[year_begin & year_end]

ac_ten_yr_filtered_weather.head()

,latitude,longitude,State,State Abbr,region,sub-region,country,year,max_temp_avg,min_temp_avg,daylight_hr_avg,rain_avg,snowfall_avg
3988,32.794376,-86.809296,Alabama,AL,South,East South Central,US,2011,62.659161,45.350056,12.206070,0.092383,0.110627
4353,32.794376,-86.809296,Alabama,AL,South,East South Central,US,2012,64.784424,47.059196,12.198280,0.076628,0.094328
4719,32.794376,-86.809296,Alabama,AL,South,East South Central,US,2013,61.384857,44.326859,12.206061,0.088448,0.101590
5084,32.794376,-86.809296,Alabama,AL,South,East South Central,US,2014,61.445709,43.903133,12.205998,0.085033,0.108134
5449,32.794376,-86.809296,Alabama,AL,South,East South Central,US,2015,63.295246,45.951653,12.206015,0.090867,0.094559


In [23]:
ac_weather_energy_merged = pd.merge(ac_ten_yr_filtered_weather, energy_data, on = 'State Abbr', how = 'left')
ac_weather_energy_merged

,latitude,longitude,State,State Abbr,region,sub-region,country,year,max_temp_avg,min_temp_avg,...,2011(W),2012(W),2013(W),2014(W),2015(W),2016(W),2017(W),2018(W),2019(W),2020(W)
0,32.794376,-86.809296,Alabama,AL,South,East South Central,US,2011,62.659161,45.350056,...,0,0,0,0,0,0,0,0,0,0
1,32.794376,-86.809296,Alabama,AL,South,East South Central,US,2012,64.784424,47.059196,...,0,0,0,0,0,0,0,0,0,0
2,32.794376,-86.809296,Alabama,AL,South,East South Central,US,2013,61.384857,44.326859,...,0,0,0,0,0,0,0,0,0,0
3,32.794376,-86.809296,Alabama,AL,South,East South Central,US,2014,61.445709,43.903133,...,0,0,0,0,0,0,0,0,0,0
4,32.794376,-86.809296,Alabama,AL,South,East South Central,US,2015,63.295246,45.951653,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,42.776798,-107.440826,Wyoming,WY,West,Mountain,US,2016,63.879883,46.840820,...,44809,41576,42298,41894,34990,40505,39785,36912,37045,48329
506,42.776798,-107.440826,Wyoming,WY,West,Mountain,US,2017,63.423321,46.237774,...,44809,41576,42298,41894,34990,40505,39785,36912,37045,48329
507,42.776798,-107.440826,Wyoming,WY,West,Mountain,US,2018,62.477726,45.290493,...,44809,41576,42298,41894,34990,40505,39785,36912,37045,48329
508,42.776798,-107.440826,Wyoming,WY,West,Mountain,US,2019,62.175228,44.758846,...,44809,41576,42298,41894,34990,40505,39785,36912,37045,48329


In [24]:
ac_weather_energy_merged.to_csv("annual_all_weather_and_energy.csv")
ac_weather_energy_merged.to_excel("annual_all_weather_and_energy.xlsx")

PermissionError: [Errno 13] Permission denied: 'annual_all_weather_and_energy.csv'

In [ ]:
# #FIVE YEAR FILTER 
# filter_start = '2016-01-01'
# filter_end = '2020-12-31' 

# # month_filter_start = 1
# # month_filter_end = 12

# year_begin = (weather_no_dups['Date'] >= filter_start) #(weather_no_dups['month'] >= month_filter_start)

# year_end = (weather_no_dups['Date'] <= filter_end) #(weather_no_dups['month'] <= month_filter_end)

# # Apply the filters
# five_building_and_weather = weather_no_dups.loc[year_begin & year_end]

# five_building_and_weather.head()

In [ ]:
# building = pd.read_csv("C:/Github/Group3_Project4/pete files/pete_resources/regional_home_data.csv")
# building = building.rename(columns = {'Region': 'region'})
# building.head()

In [ ]:
# building_and_weather = pd.merge(ten_yr_filtered_weather, building, on = 'region', how = 'left')
# building_and_weather.head()

In [ ]:
# ten_yr_building_and_weather = building_and_weather.drop(columns = ['Unnamed: 0', 'Total Residential Buildings (millions)', 'Before 1950 (millions)', 
#                                                              '1950-1959 (millions)', '1960-1969 (millions)', '1970-1979 (millions)',
#                                                              '1980-1989 (millions)', '1990-1999 (millions)', '2000-2009 (millions)'])
# ten_yr_building_and_weather.head()

In [ ]:
# ten_yr_building_and_weather[['month', 'year','max_temp_avg', 'min_temp_avg', 'daylight_hr_avg', 'rain_avg', 'snowfall_avg', '2010-2015 (millions)', '2016-2020 (millions)']] = (
#     ten_yr_building_and_weather[['month', 'year','max_temp_avg', 'min_temp_avg', 'daylight_hr_avg', 'rain_avg', 'snowfall_avg', '2010-2015 (millions)', '2016-2020 (millions)']]
#     .astype(float)
#     )

# ten_yr_building_and_weather[['month' 'year']] = pd.to_datetime(ten_yr_building_and_weather[['month' 'year']])

In [ ]:

# ten_yr_building_and_weather.to_excel("pete_resources/10yr_home_and_weather.xlsx")
# ten_yr_building_and_weather.to_csv("pete_resources/10yr_home_and_weather.csv")
# ten_yr_building_and_weather.dtypes

In [ ]:
# five_building_and_weather = pd.merge(five_yr_filtered_weather, building, on = 'region', how = 'left')
# five_building_and_weather.head()

In [ ]:
# five_building_and_weather = five_building_and_weather.drop(columns = ['Unnamed: 0', 'Total Residential Buildings (millions)', 'Before 1950 (millions)', 
#                                                              '1950-1959 (millions)', '1960-1969 (millions)', '1970-1979 (millions)',
#                                                              '1980-1989 (millions)', '1990-1999 (millions)', '2000-2009 (millions)',
#                                                               '2010-2015 (millions)'])
# five_building_and_weather.head()

In [ ]:
# five_building_and_weather.to_csv("pete_resources/5yr_home_and_weather.csv")

In [ ]:
# all_building_weather = pd.merge(weather_no_dups, building, on = 'region', how = 'left')

# all_building_weather = all_building_weather.drop(columns = ['Unnamed: 0', 'Total Residential Buildings (millions)', 'Before 1950 (millions)', 
#                                                              '1950-1959 (millions)', '1960-1969 (millions)', '1970-1979 (millions)',
#                                                              '1980-1989 (millions)', '1990-1999 (millions)'])
# all_building_weather.head()

In [ ]:
# all_building_weather.to_csv("pete_resources/all_home_and_weather.csv")